In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np

import re
import ast
import html
import glob
import spacy
import emoji
import string
import warnings

from datetime import datetime
from string import punctuation

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
import plotly.express as px
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.decomposition import PCA
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer

warnings.filterwarnings('ignore', 'SelectableGroups dict interface')

/Users/aayushagrawal/anaconda3/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/aayushagrawal/anaconda3/lib/python3.7/site-packages/plotly/express/_doc.py:503: DeprecationWarning:

inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()

/Users/aayushagrawal/anaconda3/lib/python3.7/site-packages/plotly/express/_doc.py:503: DeprecationWarning:

inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()

/Users/aayushagrawal/anaconda3/lib/python3.7/site-packages/plotly/express/_doc.py:503: DeprecationWarning:

inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()

/Users/aayushagrawal/anaconda3/lib/python3.7/site-packages/plotly/express/_doc.py:503: DeprecationWarning:

inspect.getargs

In [3]:
nlp = spacy.load("en_core_web_lg")

In [9]:
frame = pd.read_csv("emails_dir/email_30-03-2023-08-54-44_30-03-2023-09-05-28.csv")
frame = frame[frame["id"] != "id"].reset_index(drop = True)
frame.shape

(1463, 8)

In [10]:
def search_pattern(x, pattern = '(.+?)<'):
    try:
        found = re.search(pattern, x).group(1)
    except AttributeError:
        found = np.nan
    return(found) 

In [11]:
## Data Preprocessing

frame["headers"] = frame["payload"].apply(lambda x:ast.literal_eval(x).get("headers"))
frame["headers_keys"] = frame["headers"].apply(lambda x: pd.DataFrame(x)["name"].str.lower().values.tolist())
frame["from"] = frame["headers"].apply(
    lambda x:dict(
        zip(pd.DataFrame(x)["name"].str.lower().values.tolist(), 
            pd.DataFrame(x)["value"].values.tolist()))["from"]
)

frame["from_email"] = frame["from"].apply(lambda x:search_pattern(x, "<(.+?)>"))
frame["from_name"] = frame["from"].apply(lambda x:search_pattern(x, "(.+?)<"))

frame_sub = frame[["subject" in elem for elem in frame["headers_keys"]]].reset_index(drop = True)
frame_sub["subject"] = frame_sub["headers"].apply(
    lambda x:dict(zip(pd.DataFrame(x)["name"].str.lower().values.tolist(), 
                      pd.DataFrame(x)["value"].values.tolist()))["subject"]
)

frame_sub["from_email"] = frame_sub["from_email"].fillna(frame_sub["from"])
frame_sub["from_name"] = frame_sub["from_name"].fillna(frame_sub["from"])
frame_sub["internalDateUsual"] = frame_sub["internalDate"].apply(lambda x:datetime.utcfromtimestamp(float(x)/1000).strftime('%Y-%m-%d %H:%M:%S'))
frame_sub["payload"] = frame_sub["payload"].apply(lambda x:ast.literal_eval(x))

frame_sub["subject"] = frame_sub["subject"].fillna("")
frame_sub["snippet"] = frame_sub["snippet"].fillna("")

frame_sub["emoji"] = frame_sub.apply(lambda x:[emoji.demojize(elem) for elem in emoji.distinct_emoji_list(x["subject"] + x["snippet"])], axis = 1)
frame_sub["subject-snippet"] = frame_sub["subject"] + ". " + frame_sub["snippet"]

frame_sub.head(2)

,id,threadId,labelIds,snippet,payload,sizeEstimate,historyId,internalDate,headers,headers_keys,from,from_email,from_name,subject,internalDateUsual,emoji,subject-snippet
0,187307ea1922964e,187307ea1922964e,"['CATEGORY_PROMOTIONS', 'UNREAD', 'INBOX']",Redeem on your favorite audiobook ‌ ‌ ‌ ‌ ‌ ‌ ...,"{'partId': '', 'mimeType': 'multipart/alternat...",73786,10498173,1680145814000,"[{'name': 'Delivered-To', 'value': 'aagrawal52...","[delivered-to, received, x-google-smtp-source,...","""Audible.in"" <newsletters@audible.in>",newsletters@audible.in,"""Audible.in""",Special offer | ₹200 voucher for you,2023-03-30 03:10:14,[],Special offer | ₹200 voucher for you. Redeem o...
1,187307919b31e701,187307919b31e701,"['IMPORTANT', 'CATEGORY_UPDATES', 'INBOX']","Dear Aayush, Your order is arriving early! You...","{'partId': '', 'mimeType': 'multipart/alternat...",58448,10498300,1680145455000,"[{'name': 'Delivered-To', 'value': 'aagrawal52...","[delivered-to, received, x-google-smtp-source,...",Shiprocket <info@net.shiprocket.in>,info@net.shiprocket.in,Shiprocket,"Aayush Agrawal, your Firki Wholesale Private L...",2023-03-30 03:04:15,[],"Aayush Agrawal, your Firki Wholesale Private L..."


In [12]:
frame_sub["from_email_domain"] = frame_sub["from_email"].apply(lambda x:x.split("@")[1])
frame_sub.head()

,id,threadId,labelIds,snippet,payload,sizeEstimate,historyId,internalDate,headers,headers_keys,from,from_email,from_name,subject,internalDateUsual,emoji,subject-snippet,from_email_domain
0,187307ea1922964e,187307ea1922964e,"['CATEGORY_PROMOTIONS', 'UNREAD', 'INBOX']",Redeem on your favorite audiobook ‌ ‌ ‌ ‌ ‌ ‌ ...,"{'partId': '', 'mimeType': 'multipart/alternat...",73786,10498173,1680145814000,"[{'name': 'Delivered-To', 'value': 'aagrawal52...","[delivered-to, received, x-google-smtp-source,...","""Audible.in"" <newsletters@audible.in>",newsletters@audible.in,"""Audible.in""",Special offer | ₹200 voucher for you,2023-03-30 03:10:14,[],Special offer | ₹200 voucher for you. Redeem o...,audible.in
1,187307919b31e701,187307919b31e701,"['IMPORTANT', 'CATEGORY_UPDATES', 'INBOX']","Dear Aayush, Your order is arriving early! You...","{'partId': '', 'mimeType': 'multipart/alternat...",58448,10498300,1680145455000,"[{'name': 'Delivered-To', 'value': 'aagrawal52...","[delivered-to, received, x-google-smtp-source,...",Shiprocket <info@net.shiprocket.in>,info@net.shiprocket.in,Shiprocket,"Aayush Agrawal, your Firki Wholesale Private L...",2023-03-30 03:04:15,[],"Aayush Agrawal, your Firki Wholesale Private L...",net.shiprocket.in
2,187305d465cacaa2,187305d465cacaa2,"['Label_6876589031423072572', 'IMPORTANT', 'CA...","Dear Card Member, Thank you for using your HDF...","{'partId': '', 'mimeType': 'multipart/alternat...",12648,10497801,1680143630000,"[{'name': 'Delivered-To', 'value': 'aagrawal52...","[delivered-to, received, x-google-smtp-source,...",HDFC Bank InstaAlerts <alerts@hdfcbank.net>,alerts@hdfcbank.net,HDFC Bank InstaAlerts,Alert : Update on your HDFC Bank Credit Card,2023-03-30 02:33:50,[],Alert : Update on your HDFC Bank Credit Card....,hdfcbank.net
3,187305d1717d41ac,187305d1717d41ac,"['Label_4986213670456924233', 'IMPORTANT', 'CA...",Stay updated with what you read! Premium Inves...,"{'partId': '', 'mimeType': 'multipart/mixed', ...",79245,10497924,1680143619000,"[{'name': 'Delivered-To', 'value': 'aagrawal52...","[delivered-to, received, x-google-smtp-source,...",Mint <info@hindustantimesnewletter.com>,info@hindustantimesnewletter.com,Mint,Handpicked stories - stay updated with what yo...,2023-03-30 02:33:39,[],Handpicked stories - stay updated with what yo...,hindustantimesnewletter.com
4,1873037f29e0273c,1873037f29e0273c,"['Label_5479026944618646578', 'CATEGORY_UPDATE...","Registered Office: 3rd Floor, Maker Chambers I...","{'partId': '', 'mimeType': 'multipart/alternat...",33551,10497982,1680140904000,"[{'name': 'Delivered-To', 'value': 'aagrawal52...","[delivered-to, received, x-google-smtp-source,...",RELIANCE INDUSTRIES LIMITED <investor.relation...,investor.relations@ril.com,RELIANCE INDUSTRIES LIMITED,Meeting of the Equity Shareholders of Reliance...,2023-03-30 01:48:24,[],Meeting of the Equity Shareholders of Reliance...,ril.com


In [13]:
frame_sub.groupby("from_email_domain").size().sort_values(ascending = False).head(50)

from_email_domain
gmail.com                       511
google.com                       65
hdfcbank.net                     64
nsdl.co.in                       61
linkedin.com                     40
cdslstatement.com                35
jupiter.money                    31
isical.ac.in                     25
livemint.net                     24
citicorp.com                     21
procam.in                        18
cdslindia.co.in                  17
fcbayern.com                     17
swiggy.in                        14
email.apple.com                  14
deshaw.com                       13
alphonso.tv                      12
isibang.ac.in                    11
email.vestedfinance.com          11
amazon.com                       10
twitter.com                      10
actuariesindia.org               10
easemytrip.com                   10
isid.ac.in                        9
upwork.com                        9
medium.com                        9
dhl.com                           8
e.fanatics

In [14]:
frame_sub[frame_sub["from_email_domain"] == "hdfcbank.net"].groupby("from_email").size().sort_values(ascending = False)

from_email
alerts@hdfcbank.net                       29
hdfcbanksmartstatement@hdfcbank.net       15
feedback@hdfcbank.net                     12
Emailstatements.cards@hdfcbank.net         2
HDFCBANK.CREDITCARD@hdfcbank.net           2
information@hdfcbank.net                   2
CreditCardOps.Maintenance@hdfcbank.net     1
customerinfo@hdfcbank.net                  1
dtype: int64

In [17]:
frame_sub[frame_sub["from_email_domain"] == "gmail.com"].groupby("from_email").size().sort_values(ascending = False)[20:40]

from_email
022subhadip@gmail.com           6
kiranmoy.das@gmail.com          6
tejaswitri1996@gmail.com        6
depiyal7@gmail.com              5
keyurpethad1996@gmail.com       5
saurabh.messi.sood@gmail.com    5
srishti.rs@gmail.com            4
agrawal.amol2010@gmail.com      4
ahanabasistha@gmail.com         4
amitapal59@gmail.com            4
paulsounak96@gmail.com          4
pallavijindal07@gmail.com       4
ananya03pari@gmail.com          4
abhijitsen2009@gmail.com        4
soumendu041@gmail.com           4
adityajh39@gmail.com            4
isiaa2007@gmail.com             3
indra.chastity@gmail.com        3
talukdar.himasish@gmail.com     3
projecthelp143@gmail.com        3
dtype: int64

In [19]:
frame_sub[frame_sub["from_email_domain"] == "linkedin.com"].groupby("from_email").size().sort_values(ascending = False)

from_email
jobalerts-noreply@linkedin.com    32
messages-noreply@linkedin.com      3
inmail-hit-reply@linkedin.com      2
jobs-noreply@linkedin.com          2
hit-reply@linkedin.com             1
dtype: int64